In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
path = "OpenGVLab/InternVL3-1B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()


/home/vipin/miniconda3/envs/internvl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


FlashAttention2 is not installed.


In [ ]:
import torch
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from PIL import Image
from transformers import AutoModel, AutoTokenizer

# ==== CONFIG ====
# If you downloaded locally:
model_path = "pretrained/InternVL3-1B"
# Or from hub (needs net): "OpenGVLab/InternVL3-1B"

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ])
    return transform

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j)
        for n in range(min_num, max_num + 1)
        for i in range(1, n + 1)
        for j in range(1, n + 1)
        if i * j <= max_num and i * j >= min_num
    )
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
        best_ratio_diff = float('inf')
        best_ratio = (1, 1)
        area = width * height
        for ratio in target_ratios:
            target_aspect_ratio = ratio[0] / ratio[1]
            ratio_diff = abs(aspect_ratio - target_aspect_ratio)
            if ratio_diff < best_ratio_diff:
                best_ratio_diff = ratio_diff
                best_ratio = ratio
            elif ratio_diff == best_ratio_diff:
                if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                    best_ratio = ratio
        return best_ratio

    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size
    )

    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size,
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)

    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(im) for im in images]
    pixel_values = torch.stack(pixel_values)  # [num_tiles, 3, H, W]
    return pixel_values

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # fp16 or bf16 – pick ONE
    dtype = torch.float16  # or torch.bfloat16

    print("Loading model...")
    model = AutoModel.from_pretrained(
        model_path,
        torch_dtype=dtype,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        device_map="auto",      # let HF place on GPU
    ).eval()

    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True,
        use_fast=False,
    )

    # ==== IMAGE PATH HERE ====
    image_path = "data/DriveLM/drivelm_nus_imgs_train/nuscenes/samples/CAM_BACK/n008-2018-09-18-14-54-39-0400__CAM_BACK__1537297457937558.jpg"  # change to your image
    pixel_values = load_image(image_path, max_num=12).to(dtype).to(device)

    generation_config = dict(max_new_tokens=256, do_sample=True)

    # ---- Pure text chat ----
    question = "Hello, who are you?"
    response, history = model.chat(
        tokenizer, None, question, generation_config,
        history=None, return_history=True
    )
    print(f"\n[TEXT] {question}\n>> {response}")

    # ---- Single image chat ----
    question = "<image>\nPlease describe the image shortly."
    response = model.chat(
        tokenizer, pixel_values, question, generation_config
    )
    img = open(image_path)
    print(f"\n[IMAGE] {question}\n>> {response}")

if __name__ == "__main__":
    main()


Using device: cuda
Loading model...


OSError: work_dirs/internvl_chat_v3/internvl3_1b_dynamic_res_2nd_finetune_full/checkpoint-170080 does not appear to have a file named configuration_internvl_chat.py. Checkout 'https://huggingface.co/work_dirs/internvl_chat_v3/internvl3_1b_dynamic_res_2nd_finetune_full/checkpoint-170080/tree/main' for available files.

In [7]:
import json
from pathlib import Path

src = Path("data/DriveLM/drivelm_val.jsonl")          # your original file
dst = Path("data/DriveLM/drivelm_val_fixed.jsonl")    # new file

prefix = "nuscenes/samples/"

with src.open() as fin, dst.open("w") as fout:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        img = obj.get("image", "")
        if img.startswith(prefix):
            obj["image"] = img[len(prefix):]  # remove "nuscenes/samples/"
        fout.write(json.dumps(obj) + "\n")

print("Done, wrote:", dst)


Done, wrote: data/DriveLM/drivelm_val_fixed.jsonl
